In [1]:
from Environment import *
import matplotlib.pyplot as plt
from Model.Evaluator.GraphEvaluator import *
from Learner.BruteForce import *
from Learner.UCB.UCB_Step4 import *
from Model.ConfigurationParametersAverage import mergeUserClasses
from scipy.stats import gamma
import time

In [2]:
#files = ['./Configs/config1.json', './Configs/config2.json','./Configs/config3.json', './Configs/configDump.json', './Configs/configuration4.json', './Configs/configuration5.json']
files = ['./Configs/config2.json']
env = []
config_margins = []
optimal_arms = []
conv_rates = []
prod_lists = []
click_probs = []
lambdas = []
alphas = []
units_means = []
clairvoyant_opt_rew = []
secondary = []
actual_unit_mean = []

for i in range(0, len(files)):
    env.append(Environment(config_path=files[i]))
    config = mergeUserClasses([files[i]], False)[0]
    config_margins.append(config["marginsPerPrice"])
    optimal_arms.append(config["optimalConfig"])
    conv_rates.append(config["conversionRateLevels"])
    prod_lists.append(config["productList"])
    print("ProdList={}, Alphas={}, ConvRates={}".format(len(config["productList"]),len(config["alphas"]),len(config["conversionRateLevels"])))
    click_probs.append(config["click_prob"])
    lambdas.append(config['lambda_p'])
    alphas.append(config["alphas"])
    clairvoyant_opt_rew.append(config["optimalMargin"])
    units_means.append(config["units_mean"])
    actual_unit_mean.append(config["actual_units_mean"])

print(alphas[0])

ProdList=5, Alphas=5, ConvRates=5
[0.2642857142857143, 0.22142857142857142, 0.21714285714285714, 0.14857142857142858, 0.14857142857142858]


In [3]:
#Brute force to find best configuration
n_experiments = 1024

for i in range(0, len(env)):
    maxMargin = -1
    optConf = [0, 0, 0, 0, 0]
    secondBestMargin = -1
    secondOptimal = [0, 0, 0, 0, 0]
    learner = BruteForce()

    for j in range(0, n_experiments):

        pulledArm = learner.pull_arm()

        armMargins = []
        armConvRates = []

        for k in range(0, len(pulledArm)):
            armMargins.append(config_margins[i][k][pulledArm[k]])
            armConvRates.append(conv_rates[i][k][pulledArm[k]])

        graphEval = GraphEvaluator(products_list=prod_lists[i], click_prob_matrix=click_probs[i],
                                   lambda_prob=lambdas[i],
                                   alphas=alphas[i], conversion_rates=armConvRates, margins=armMargins,
                                   units_mean=actual_unit_mean[i], verbose=False, convert_units=False)

        TotMargin = graphEval.computeMargin()

        learner.update(TotMargin)
        if TotMargin > maxMargin:
            maxMargin = TotMargin
            optConf = pulledArm
        if TotMargin < maxMargin and TotMargin > secondBestMargin and pulledArm != secondOptimal:
            secondBestMargin = TotMargin
            secondOptimal = pulledArm

    config_name = files[i][files[i].rfind('/')-len(files[i])+1:]
    print(config_name, "optimal is: ", optConf, maxMargin)
    print(config_name, "second optimal is: ", secondOptimal, secondBestMargin)

config2.json optimal is:  [1, 1, 1, 1, 1] 7.467142857142857
config2.json second optimal is:  [1, 1, 1, 1, 2] 7.3185714285714285


In [4]:
# Check difference between Graph eval e environment margin
n_experiments = 100
n_user = [100, 70, 80, 95]

for i in range(0, len(env)):
    maxMargin = -1
    optConf = [0, 0, 0, 0, 0]
    secondBestMargin = -1
    secondOptimal = [0, 0, 0, 0, 0]
    learner = BruteForce()
    rmse = 0

    for j in range(0, n_experiments):
        armMargins = []
        armConvRates = []
        pulledArm = learner.pull_arm()
        #print(pulledArm)
        for k in range(0, len(pulledArm)):
            armMargins.append(config_margins[i][k][pulledArm[k]])
            armConvRates.append(conv_rates[i][k][pulledArm[k]])

        graphEval = GraphEvaluator(products_list=prod_lists[i], click_prob_matrix=click_probs[i], lambda_prob=lambdas[i],
                    alphas=alphas[i], conversion_rates=armConvRates, margins=armMargins, units_mean=actual_unit_mean[i], verbose=False, convert_units=False)

        tempMargin = graphEval.computeMargin()

        env[i].setPriceLevels(pulledArm)
        mc_margin = 0
        start = time.time()
        for exp in range(0, 100):
            interactions = env[i].round()
            mc = 0
            for k in range(0, len(interactions)):
                mc = mc + interactions[k].linearizeMargin(config_margins[i])
            mc_margin += mc / len(interactions)

        mc_margin = mc_margin / 100 # (100*n_user[i])
        #print(tempMargin, mc_margin)
        rmse += (tempMargin - mc_margin)**2

        learner.update(tempMargin)
        if tempMargin > maxMargin:
            maxMargin = tempMargin
            optConf = pulledArm
        if tempMargin < maxMargin and tempMargin > secondBestMargin and pulledArm != secondOptimal:
            secondBestMargin = tempMargin
            secondOptimal = pulledArm

    config_name = files[i][files[i].rfind('/')-len(files[i])+1:]
    rmse = math.sqrt(rmse/n_experiments)
    print("RMSE: ", rmse)
    print("% error: ", (rmse/maxMargin)*100)
    print(config_name, "optimal is: ", optConf, maxMargin)
    print(config_name, "second optimal is: ", secondOptimal, secondBestMargin)

RMSE:  2.4948686294310596
% error:  35.95651720407127
config2.json optimal is:  [0, 1, 1, 1, 1] 6.938571428571429
config2.json second optimal is:  [0, 1, 1, 1, 2] 6.79


In [10]:
n_experiments = 1024

for i in range(0, len(env)):
    maxMargin = -1
    optConf = [0, 0, 0, 0, 0]
    secondBestMargin = -1
    secondOptimal = [0, 0, 0, 0, 0]
    learner = BruteForce()

    for j in range(0, n_experiments):

        pulledArm = learner.pull_arm()

        TotMargin = 0
        env[i].setPriceLevels(pulledArm)
        for iter in range(0, 100):
            rew = env[i].round()
            oldTotMargin = TotMargin
            for e in rew:
                if e.bought:
                    TotMargin += config_margins[i][e.product][pulledArm[e.product]] * e.units
            #print(TotMargin-oldTotMargin)


        TotMargin = TotMargin / 100

        learner.update(TotMargin)
        if TotMargin > maxMargin:
            maxMargin = TotMargin
            optConf = pulledArm
        if TotMargin < maxMargin and TotMargin > secondBestMargin and pulledArm != secondOptimal:
            secondBestMargin = TotMargin
            secondOptimal = pulledArm

    config_name = files[i][files[i].rfind('/')-len(files[i])+1:]
    print(config_name, "optimal is: ", optConf, maxMargin)
    print(config_name, "second optimal is: ", secondOptimal, secondBestMargin)

KeyboardInterrupt: 

In [ ]:
# Compute actual mean of products

units_means = [[0.75, 1.25, 1.6, 2.1, 0.94], #Conf1 - conf2 - conf3 - confDump
               [1.5, 1.25, 1.6, 1.9, 1.5], #Conf4
               [1.8, 1.5, 1.8, 1.9, 1.9], #Conf4
               [1.6, 1.9, 1.2, 1.5, 1.9], #Conf4
               [0.75, 1.25, 1.6, 2.1, 0.94],#Conf5
               [1.25, 1.25, 2.5, 2.1, 1], # Conf5
               [0.94, 1.45, 1.2, 2.0, 1.5]] #Conf5
for units_mean in units_means:
    actual_means = []
    for i in range(0,len(units_mean)):
        # empiric_mean = np.ceil(np.random.gamma(units_mean[i], 1, size=1000000)).mean()
        # New method using cumulative distribution, difference empiric and theoretic < 0.01
        significant = True
        theoretic_mean = 0
        num = 1
        while significant:
            t = (gamma.cdf(num, a=units_mean[i]) - gamma.cdf(num - 1, a=units_mean[i])) * num
            theoretic_mean += t
            num += 1
            if t < 0.01:
                significant = False
                actual_means.append(int(theoretic_mean*100) / 100)
    print(actual_means)